# OCT Image Denoising


Author: Xiaolong Du

### Autoencoder: denoising oct images

Removing noise from oct images.

In [0]:
import torch
import torch.cuda as cuda
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import time
import os

from torch.autograd import Variable

# Torchvision module contains various utilities, classes, models and datasets 
# used towards computer vision usecases
from torchvision import datasets
from torchvision import transforms

from torch.nn import functional as F

**Create the datasets**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.


In [0]:
transform=transforms.Compose([transforms.ToTensor()])
s = time.time()

#cifar10_train = datasets.CIFAR10('./data.cifar10', train=True, download=True, transform=transform)
#cifar10_valid = datasets.CIFAR10('./data.cifar10', train=False, download=True, transform=transform)
cifar10_train = datasets.MNIST('./dataMNIST', train=True, download=True, transform=transform)
cifar10_valid = datasets.MNIST('./dataMNIST', train=False, download=True, transform=transform)
e = time.time()
print(e-s)

**Utility to display the original, noisy and denoised image**

In [0]:
def show_img(orig, noisy, denoised):
    fig=plt.figure()
    
    #orig = orig.swapaxes(0, 1).swapaxes(1, 2)
    #noisy = noisy.swapaxes(0, 1).swapaxes(1, 2)
    #denoised = denoised.swapaxes(0, 1).swapaxes(1, 2)
    orig = orig.squeeze()
    noisy = noisy.squeeze()
    denoised = denoised.squeeze()
    
    # Normalize for display purpose
    orig     = (orig - orig.min()) / (orig.max() - orig.min())
    noisy    = (noisy - noisy.min()) / (noisy.max() - noisy.min())
    denoised = (denoised - denoised.min()) / (denoised.max() - denoised.min())
    #print(orig.size())
    
    fig.add_subplot(1, 3, 1, title='Original')
    plt.imshow(orig,cmap='gray')
    
    fig.add_subplot(1, 3, 2, title='Noisy')
    plt.imshow(noisy,cmap = 'gray')
    
    fig.add_subplot(1, 3, 3, title='Denoised')
    plt.imshow(denoised,cmap = 'gray')
    
    fig.subplots_adjust(wspace = 0.5)
    plt.show()
    
# To test


**Some hyper parameters**

In [0]:
batch_size = 250 # Reduce this if you get out-of-memory error
learning_rate = 0.001
noise_level = 0.1

**Create the dataloader**

In [0]:
cifar10_train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=10)
cifar10_valid_loader = torch.utils.data.DataLoader(cifar10_valid, batch_size=batch_size, shuffle=True, num_workers=10)

**The Denoising Autoencoder**

In [0]:
class DenoisingAutoencoder(nn.Module):
    
    def __init__(self):
    
        super(DenoisingAutoencoder, self).__init__()
                                                            # 32 x 32 x 3 (input)

        self.conv1e = nn.Conv2d(1, 24, 3, padding=2)        # 30 x 30 x 24
        self.conv2e = nn.Conv2d(24, 48, 3, padding=2)       # 28 x 28 x 48
        self.conv3e = nn.Conv2d(48, 96, 3, padding=2)       # 26 x 26 x 96
        self.conv4e = nn.Conv2d(96, 128, 3, padding=2)      # 24 x 24 x 128
        self.conv5e = nn.Conv2d(128, 256, 3, padding=2)     # 22 x 22 x 256
        self.mp1e   = nn.MaxPool2d(2, return_indices=True)  # 11 x 11 x 256

        self.mp1d = nn.MaxUnpool2d(2)
        self.conv5d = nn.ConvTranspose2d(256, 128, 3, padding=2)
        self.conv4d = nn.ConvTranspose2d(128, 96, 3, padding=2)
        self.conv3d = nn.ConvTranspose2d(96, 48, 3, padding=2)
        self.conv2d = nn.ConvTranspose2d(48, 24, 3, padding=2)
        self.conv1d = nn.ConvTranspose2d(24, 1, 3, padding=2)
        
    
    def forward(self, x):
        # Encoder
        x = self.conv1e(x)
        x = F.relu(x)
        x = self.conv2e(x)
        x = F.relu(x)
        x = self.conv3e(x)
        x = F.relu(x)
        x = self.conv4e(x)
        x = F.relu(x)
        x = self.conv5e(x)
        x = F.relu(x)
        x, i = self.mp1e(x)
        
         # Decoder
        x = self.mp1d(x, i)
        x = self.conv5d(x)
        x = F.relu(x)
        x = self.conv4d(x)
        x = F.relu(x)
        x = self.conv3d(x)
        x = F.relu(x)
        x = self.conv2d(x)
        x = F.relu(x)
        x = self.conv1d(x)
        x = F.relu(x)
        
        return x


In [0]:
autoencoder = DenoisingAutoencoder().cuda()
parameters = list(autoencoder.parameters())
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(parameters, lr=learning_rate)

In [0]:
train_loss = []
valid_loss = []


for i in range(10):
    
    # Let's train the model
    s = time.time()
    
    total_loss = 0.0
    total_iter = 0
    autoencoder.train()
    for image, label in cifar10_train_loader:
        
        noise = torch.randn(image.shape[0], 1, 28, 28) * noise_level
        image_n = torch.add(image, noise)
        
        image = Variable(image).cuda()
        image_n = Variable(image_n).cuda()

        optimizer.zero_grad()
        output = autoencoder(image_n)
        
        loss = loss_func(output, image)
        loss.backward()
        optimizer.step()
        
        total_iter += 1
        total_loss += loss.item()
        
    # Let's record the validation loss
    
    total_val_loss = 0.0
    total_val_iter = 0
    autoencoder.eval()
    for image, label in cifar10_valid_loader:
        
        noise = torch.randn(image.shape[0], 1, 28, 28) * noise_level
        image_n = torch.add(image, noise)
        
        image = Variable(image).cuda()
        image_n = Variable(image_n).cuda()
        
        output = autoencoder(image_n)
        loss = loss_func(output, image)
        
        total_val_iter += 1
        total_val_loss += loss.item()
        
        
    # Let's visualize the first image of the last batch in our validation set
    orig = image[0].cpu()
    noisy = image_n[0].cpu()
    denoised = output[0].cpu()

    orig = orig.data.numpy()
    noisy = noisy.data.numpy()
    denoised = denoised.data.numpy()

    print("Iteration ", i+1)
    show_img(orig, noisy, denoised)
    
    train_loss.append(total_loss / total_iter)
    valid_loss.append(total_val_loss / total_val_iter)
    e = time.time()
    print(e-s)

In [0]:
# Save the model
torch.save(autoencoder.state_dict(), "./gdrive/My Drive/autoencoder.pth")

In [0]:
fig = plt.figure(figsize=(10, 7))
plt.plot(train_loss, label='Train loss')
plt.plot(valid_loss, label='Validation loss')
plt.legend()
plt.show()

###  Single image denoising

In [0]:
import random

img, _ = random.choice(cifar10_valid)
img    = img.resize_((1, 1, 28, 28))
noise  = torch.randn((1, 1, 28, 28)) * noise_level
img_n  = torch.add(img, noise)

img_n = Variable(img_n).cuda()
denoised = autoencoder(img_n)


show_img(img[0].numpy(), img_n[0].data.cpu().numpy(), denoised[0].data.cpu().numpy())